In [14]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
from interlab.queries import query_for_json, query_model
import langchain_community.chat_models
from treetrace import TracingNode
from interlab_zoo.persona.personas import Factory
from interlab_zoo.persona.contacts import link
from interlab_zoo.persona.channels import ChatChannel
import langchain

dotenv.load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [2]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
set_llm_cache(InMemoryCache())


In [3]:
factory = Factory.from_yaml("audience.yaml")

In [4]:
marie = factory.create_persona("Maria-gpt35")
lilly = factory.create_persona("Lilly-gpt35")

/home/ada/.cache/pypoetry/virtualenvs/interlab-cZHazbI_-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
marie.name

'Maria'

In [40]:
marie.public_description()

'Public information about Maria: age 22\nMaria is a mentor\n'

In [54]:
link([marie, lilly])

In [66]:
with TracingNode("root") as root:
    ChatChannel.query_and_send_message(marie, lilly)
root.display()

In [16]:

class Situation:

    def __init__(self, subject, factory, person_keys):
        self.factory = factory
        self.person_keys = person_keys
        self.players = [self.factory.create_persona(key, goal=f"You want to make an agreement on {subject}") for key in person_keys]        
        names = ",".join(p.name for p in self.players)
        judge_model = langchain_community.chat_models.ChatOpenAI(model="gpt-3.5-turbo")
        self.judge = interlab.actor.llm_actor.OneShotLLMActor("Judge", judge_model, "You observe the following communication:")
        self.subject = subject
        self.agreement_result = None

        @dataclass
        class JudgeAction:
            has_agreement: int | None = Field(
                description=f"Do they have an agreement? 1 if they have a agreement; otherwise 0",
            )            
            agreement_subject: str | None = Field(
                description=f"What is ${subject} they agreed on? If there is no agreement it should be null",
            )
        self.judge_action = JudgeAction

    def agreement_check(self):
        players = situation.players
        names = ",".join(p.name for p in players)
        question = f"Do all sides ({names}) made an agreement on {self.subject}? If yes, what do they agree on?"
        response = self.judge.query(question)
        prompt = (f"Question: {question}\n\nAnswer: {response}\n\n If the answer implies that there is no agreement then write <result>none</result> "
                  f"otherwise write {self.subject} between tags <result> and </result>")
        action = query_model(self.judge.model, prompt)
        print(action)
        return True
        

def resolve_direct(sitation, channel=ChatChannel, max_steps_per_player=7):
    players = situation.players
    link(players)
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message_to_all(sender, others, [situation.judge])
        if situation.agreement_check():
            return

with TracingNode("root") as root:
    situation = Situation("a specific movie name thay want to watch together", factory, ["Maria-gpt35", "Lilly-gpt35"])
    resolve_direct(situation)
    print(situation.agreement_result)
root.display()

<result>Spirited Away</result>
None
